In [2]:
import pickle
import numpy as np
import pandas as pd


df_train = pd.read_csv('train.csv')

print df_train.shape
print df_train.columns
        

(647054, 7)
Index([u'TripType', u'VisitNumber', u'Weekday', u'Upc', u'ScanCount',
       u'DepartmentDescription', u'FinelineNumber'],
      dtype='object')


In [3]:
labels = np.unique(df_train['TripType'])
renamed_labels = {}
renamed_labels_v = {}
for i, v in np.ndenumerate(labels):
    renamed_labels[i[0]] = v
    renamed_labels_v[v] = i[0]
    
print renamed_labels, renamed_labels_v

pickle.dump( renamed_labels, open( "renamed_labels.p", "wb" ) )


{0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8, 6: 9, 7: 12, 8: 14, 9: 15, 10: 18, 11: 19, 12: 20, 13: 21, 14: 22, 15: 23, 16: 24, 17: 25, 18: 26, 19: 27, 20: 28, 21: 29, 22: 30, 23: 31, 24: 32, 25: 33, 26: 34, 27: 35, 28: 36, 29: 37, 30: 38, 31: 39, 32: 40, 33: 41, 34: 42, 35: 43, 36: 44, 37: 999} {3: 0, 4: 1, 5: 2, 6: 3, 7: 4, 8: 5, 9: 6, 12: 7, 14: 8, 15: 9, 18: 10, 19: 11, 20: 12, 21: 13, 22: 14, 23: 15, 24: 16, 25: 17, 26: 18, 27: 19, 28: 20, 29: 21, 30: 22, 31: 23, 32: 24, 33: 25, 34: 26, 35: 27, 36: 28, 37: 29, 38: 30, 39: 31, 40: 32, 41: 33, 42: 34, 43: 35, 44: 36, 999: 37}


In [4]:
y = [renamed_labels_v[v] for v in df_train['TripType']]

pickle.dump( y, open( "y.p", "wb" ) )


In [5]:
weekday = pd.get_dummies(df_train['Weekday'], prefix='Weekday')
weekday = weekday.drop(weekday.columns[[0]], axis=1)
pickle.dump( weekday, open( "Weekday.p", "wb" ) )

In [6]:
print df_train.head()

   TripType  VisitNumber Weekday          Upc  ScanCount  \
0       999            5  Friday  68113152929         -1   
1        30            7  Friday  60538815980          1   
2        30            7  Friday   7410811099          1   
3        26            8  Friday   2238403510          2   
4        26            8  Friday   2006613744          2   

   DepartmentDescription  FinelineNumber  
0     FINANCIAL SERVICES            1000  
1                  SHOES            8931  
2          PERSONAL CARE            4504  
3  PAINT AND ACCESSORIES            3565  
4  PAINT AND ACCESSORIES            1017  


In [7]:
pickle.dump( df_train['ScanCount'], open( "ScanCount.p", "wb" ) )

In [ ]:
Upc = pd.get_dummies(df_train['Upc'], prefix='Upc')
Upc = Upc.drop(Upc.columns[[0]], axis=1)
pickle.dump( Upc, open( "Upc.p", "wb" ) )

In [8]:
print len(np.unique(df_train['FinelineNumber']))

9324


In [ ]:
FinelineNumber = pd.get_dummies(df_train['FinelineNumber'], prefix='Upc')
FinelineNumber = FinelineNumber.drop(FinelineNumber.columns[[0]], axis=1)
pickle.dump( FinelineNumber, open( "Upc.p", "wb" ) )

In [8]:
#pickle.dump( df_train['DepartmentDescription'], open( "DepartmentDescription.p", "wb" ) )
len(np.unique(df_train['DepartmentDescription']))

/Users/atulkumar/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


69

In [9]:

desc = df_train['DepartmentDescription'].fillna('')
words = []
for i in range(len(desc)):
    words.append(desc[i].lower())

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = None) 

vectorizer.fit(words)




In [28]:
#train_data_features = vectorizer.transform(words)
#train_data_features_arr = train_data_features.toarray()

#vocab = vectorizer.get_feature_names()
#print vocab

#dist = np.sum(train_data_features_arr, axis=0)
#print dist
#for tag, count in zip(vocab, dist):
#    print count, tag
#train_data_features
#train_feature = np.hstack((train_data_features, weekday.as_matrix(), df_train['ScanCount'].as_matrix()))


ValueError: all the input arrays must have same number of dimensions

In [29]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit(train_data_features , y )


In [33]:
df_test = pd.read_csv('test.csv')
desct = df_test['DepartmentDescription'].fillna('')
wordst = []
for i in range(len(desct)):
    wordst.append(desct[i].lower())
    
test_data_features = vectorizer.transform(wordst)
pre = forest.predict_proba(test_data_features)

In [34]:
pre

array([[  1.02369956e-03,   1.46201941e-04,   6.43806708e-03, ...,
          1.80033895e-02,   6.78571282e-02,   4.84607817e-02],
       [  3.04541889e-04,   2.47291940e-04,   5.43807474e-03, ...,
          3.31413438e-03,   1.55350178e-02,   1.32592336e-02],
       [  3.04541889e-04,   2.47291940e-04,   5.43807474e-03, ...,
          3.31413438e-03,   1.55350178e-02,   1.32592336e-02],
       ..., 
       [  2.95192253e-03,   5.70596454e-04,   6.25030193e-03, ...,
          1.09344079e-02,   4.63493329e-02,   5.68877408e-02],
       [  2.39044277e-03,   0.00000000e+00,   0.00000000e+00, ...,
          1.62388652e-02,   4.87331440e-02,   1.66262222e-01],
       [  2.95192253e-03,   5.70596454e-04,   6.25030193e-03, ...,
          1.09344079e-02,   4.63493329e-02,   5.68877408e-02]])

In [ ]:
from __future__ import print_function

import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to sklearn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]


class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features


pipeline = Pipeline([
    # Extract the subject & body
    ('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the post's subject line
            ('subject', Pipeline([
                ('selector', ItemSelector(key='subject')),
                ('tfidf', TfidfVectorizer(min_df=50)),
            ])),

            # Pipeline for standard bag-of-words model for body
            ('body_bow', Pipeline([
                ('selector', ItemSelector(key='body')),
                ('tfidf', TfidfVectorizer()),
                ('best', TruncatedSVD(n_components=50)),
            ])),

            # Pipeline for pulling ad hoc features from post's body
            ('body_stats', Pipeline([
                ('selector', ItemSelector(key='body')),
                ('stats', TextStats()),  # returns a list of dicts
                ('vect', DictVectorizer()),  # list of dicts -> feature matrix
            ])),

        ],

        # weight components in FeatureUnion
        transformer_weights={
            'subject': 0.8,
            'body_bow': 0.5,
            'body_stats': 1.0,
        },
    )),

    # Use a SVC classifier on the combined features
    ('svc', SVC(kernel='linear')),
])

# limit the list of categories to make running this exmaple faster.
categories = ['alt.atheism', 'talk.religion.misc']
train = fetch_20newsgroups(random_state=1,
                           subset='train',
                           categories=categories,
                           )
test = fetch_20newsgroups(random_state=1,
                          subset='test',
                          categories=categories,
                          )

pipeline.fit(train.data, train.target)
y = pipeline.predict(test.data)
print(classification_report(y, test.target))

In [ ]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )


# Read the test data
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print test.shape

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in xrange(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

In [ ]:
def f(x):
     return Series(dict(A = x['A'].sum(), 
                        B = x['B'].sum(), 
                        C = "{%s}" % ', '.join(x['C'])))

In [14]: df.groupby('A').apply(f)

In [ ]:
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(n_features=1000)
f = h.transform(wc.toarray())